# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [1]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

fatal: destination path 'fraudguard' already exists and is not an empty directory.
/content/fraudguard


In [3]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

import torch

# 1. Get exact versions
pt_version = torch.__version__.split('+')[0]  # e.g., 2.5.1
cuda_version = "cu" + torch.version.cuda.replace('.', '')  # e.g., cu124
wheel_url = f"https://data.pyg.org/whl/torch-{pt_version}+{cuda_version}.html"

print(f"PyTorch: {pt_version}, CUDA: {cuda_version}")
print(f"Downloading from: {wheel_url}")

# 2. Install with visible output (force reinstall to fix broken partial installs)
!pip install --force-reinstall torch-scatter torch-sparse -f $wheel_url

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
⚠️ faiss-gpu not available, using faiss-cpu
   (Graph building on CPU, but GNN training still runs on GPU!)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 84.4 MB/s eta 0:00:00
PyTorch: 2.9.0, CUDA: cu126
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 126.7 MB/s eta 0:00:00
 

Obtaining file:///content/fraudguard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 7.6 MB/s eta 0:00:00
  Building editable for fraudguard (pyproject.toml) ... done
  Created wheel for fraudguard: filename=fraudguard-0.1.0-py3-none-any.whl size=2801 sha256=abd00cd285315796bfb55d9faae784f5c5711c89a41847e0d54c9e072446ded1
  Stored in directory: /tmp/pip-ephem-wheel-cache-2lghz6mc/wheels/c6/29/62/fb6d8d095576e7e3efddf4fdcb7dfc799af71ace273f1ee84c
Successfully built fraudguard
  Attempting uninstall: structlog
    Found existing installation: structlog 25.5.0
    Uninstalling structlog-25.5.0:
      Successfully unins


✓ Environment setup complete


In [1]:
import torch
try:
    import torch_scatter
    import torch_sparse
    import fraudguard
    print("✅ Success! Libraries are installed and loaded.")
except ImportError as e:
    print(f"❌ Still missing libraries: {e}")
    # Only if you see this error should you go back and install again.

✅ Success! Libraries are installed and loaded.


## 2️⃣ Configuration

In [5]:
import os

# ==============================================
# CONFIGURATION - UPDATE THESE PATHS AS NEEDED
# ==============================================

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 0.5      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 30
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

Data: /content/drive/MyDrive/ieee-fraud-detection
Models: /content/drive/MyDrive/fraudguard-models
Logs: /content/drive/MyDrive/fraudguard-logs

Batch size: 4096
Sample fraction: 50%


## 3️⃣ Verify GPU and FAISS

In [7]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
VRAM: 15.8 GB

✓ GNN training will run on GPU

FAISS GPUs: 0
   (Using CPU FAISS for graph building - this is OK)


## 4️⃣ Load and Preprocess Data

In [8]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

Using device: cuda

Data loaded:
  Train: 177,162
  Val: 59,054
  Test: 59,054
  Fraud rate: 3.50%


## 5️⃣ Run Full AD-RL-GNN Pipeline

We use the `FraudTrainer` class to orchestrate the full pipeline, including:
1. **AdaptiveMCD**: Intelligent majority downsampling
2. **RL Agent**: Dynamic subgraph selection (Random Walk, K-Hop, K-Ego)
3. **Graph Enhancement**: Adding semantic edges
4. **GNN Training**: CrossEntropyLoss (15x weight)

In [ ]:
from src.training.trainer import FraudTrainer
from src.utils.config import load_model_config, load_data_config
import torch

# 1. Load Configs
model_cfg = load_model_config()
data_cfg = load_data_config()

# 2. Configure for High Performance ("Our Method")
# These settings activate the components that drive Table 1 results
model_cfg.training["max_epochs"] = 30
model_cfg.adaptive_mcd["alpha"] = 0.5   # Aggressiveness of downsampling
model_cfg.rl_agent["reward_scaling"] = 2.0  # Reward for finding fraud neighbors

# 3. Initialize the Full Pipeline Trainer
# This class manages the RL Agent, MCD, and GNN together
trainer = FraudTrainer(
    model_config=model_cfg, 
    data_config=data_cfg,
    device=device
)

# 4. Run Full Training (MCD + RL + GNN)
print("🚀 Starting Full AD-RL-GNN Training...")
metrics = trainer.fit(
    train_df, 
    val_df, 
    test_df,
    max_epochs=30,
    use_mcd=True,   # Enable AdaptiveMCD
    use_rl=True     # Enable RL Agent
)

print("\nTraining Complete.")

## 6️⃣ Evaluation & Claims Verification

In [ ]:
# Evaluate on Test Set
test_metrics = trainer.evaluate()

# CV Claims Comparison
CV_CLAIMS = {
    "specificity": 98.72,
    "gmeans": 83.30,      # Baseline was 83.30, Ours 98.39 in paper, target > 70 for this implementation
    "p95_latency_ms": 100,
}

achieved_spec = test_metrics["specificity"] * 100
achieved_gmeans = test_metrics["gmeans"] * 100
# Benchmark latency
print("\nBenchmarking latency...")
perf = trainer.benchmark_latency(n_runs=100)
p95_latency = perf["p95_ms"]

print("=" * 60)
print("CV CLAIMS COMPARISON")
print("=" * 60)
print(f"| {'Metric':<20} | {'Achieved':>12} | {'Target':>12} | {'Status':>6} |")
print(f"|{'-'*22}|{'-'*14}|{'-'*14}|{'-'*8}|")

# Specificity
status_spec = "✓" if achieved_spec >= 95 else "~"
print(f"| {'Specificity':<20} | {achieved_spec:>11.2f}% | {CV_CLAIMS['specificity']:>11.2f}% | {status_spec:>6} |")

# G-Means
status_gm = "✓" if achieved_gmeans >= 70 else "~"
print(f"| {'G-Means':<20} | {achieved_gmeans:>11.2f}% | {'> 70.00':>12} | {status_gm:>6} |")

# Latency
status_lat = "✓" if p95_latency < CV_CLAIMS['p95_latency_ms'] else "✗"
print(f"| {'P95 Latency':<20} | {p95_latency:>10.1f}ms | {'<100':>10}ms | {status_lat:>6} |")


## 7️⃣ Save Model

In [ ]:
trainer.save(f"{MODELS_DIR}/fraudguard_full_pipeline.pt")
print(f"Model saved to {MODELS_DIR}/fraudguard_full_pipeline.pt")